In [10]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import numpy as np
import datetime
import data_loader
import plot_tools
import spacepy.coordinates
import spacepy.time
import spacepy.toolbox
import spacepy.omni
import spacepy.LANLstar
import spacepy.irbempy

import matplotlib.pyplot as plt
from matplotlib import colors

import importlib
importlib.reload(data_loader)
importlib.reload(plot_tools)


%matplotlib qt

In [2]:
#Parameters
#Dates
start = datetime.datetime(year = 2013, month = 3, day = 1)
end = datetime.datetime(year = 2013, month = 4, day = 1)
epoch_bin_size = datetime.timedelta(hours = 4)

#L
L_bin_size = 0.05
L_start = 3.0
L_end = 7.0

#MLT
mlt_start = 12
mlt_end = 24

#OMNI
UPDATE_SPACEPY_OMNI = False

In [3]:
satellite_names = ["metop01", "metop02", "metop03", "noaa15", "noaa16", "noaa17", "noaa18", "noaa19"]

satellites = []

for sat in satellite_names:
    
    data = data_loader.load_compressed_poes_data(satellite = sat,
                                                 start = start,
                                                 end = end)
            
    if len(data["EPOCH"]) == 0:
        continue
    
    satellites.append(data)


Loading : POES_2013_metop01_CLEAN.npz
Loading : POES_2013_metop02_CLEAN.npz
Unable to find any compressed POES data for: metop03

Data file not found: c:\Dev\Research\REPT_Enhancements_Tool\compressed_data\POES\CLEAN\metop03\POES_2013_metop03_CLEAN.npz, continuing...!
Loading : POES_2013_noaa15_CLEAN.npz
Loading : POES_2013_noaa16_CLEAN.npz
Loading : POES_2013_noaa17_CLEAN.npz
Loading : POES_2013_noaa18_CLEAN.npz
Loading : POES_2013_noaa19_CLEAN.npz


In [4]:
print(satellites[0]["EPOCH"].shape)
print(satellites[0]["ALT"].shape)
print(satellites[0]["LAT"].shape)
print(satellites[0]["LON"].shape)
print(satellites[0]["L"].shape)
print(satellites[0]["MLT"].shape)
print(satellites[0]["MEP_ELE_TEL0_FLUX_E1"].shape)
print(satellites[0]["MEP_ELE_TEL0_FLUX_E2"].shape)
print(satellites[0]["MEPED_ALPHA_0_SAT"].shape)

print(len(satellites))

(1332694,)
(1332694,)
(1332694,)
(1332694,)
(1332694,)
(1332694,)
(1332694,)
(1332694,)
(1332694,)
7


In [5]:
if UPDATE_SPACEPY_OMNI:
    spacepy.toolbox.update(omni2=True)

In [20]:
epoch = np.zeros(shape=(0))
L = np.zeros(shape=(0))
mlt = np.zeros(shape=(0))
naive_chorus = np.zeros(shape=(0))

for i, sat in enumerate(satellites):
    
    print(f"Processing satellite: {satellite_names[i]}")
    
    #Calculate Chorus
    j_40 = sat["MEP_ELE_TEL0_FLUX_E1"]
    j_130 = sat["MEP_ELE_TEL0_FLUX_E2"]

    J = j_40 - j_130
    # For some stupid reason this subtraction rarely creates negatives even though the channels are integral... :(
    J[J < 0] = np.NaN

    P = 400
    naive_chorus_intensity = J / (P * ((sat["L"] - 3)**2 + 0.03))
    naive_chorus_amplitudes = np.sqrt(naive_chorus_intensity)
      
    #Concatenate all the data
    epoch = np.concatenate((epoch, sat["EPOCH"]), axis = 0)
    L = np.concatenate((L, sat["L"]), axis = 0)
    mlt = np.concatenate((mlt, sat["MLT"]), axis = 0)
    naive_chorus = np.concatenate((naive_chorus, naive_chorus_amplitudes), axis = 0)

within_mlt_range = (mlt_start <= mlt) & (mlt < mlt_end)
within_L_range = (L_start < L) & (L < L_end)
within_epoch_range = (start < epoch) & (epoch < end)
within_all_ranges = within_mlt_range & within_L_range & within_epoch_range
finite_chorus = np.isfinite(naive_chorus)

epoch = epoch[within_all_ranges & finite_chorus]
L = L[within_all_ranges & finite_chorus]
mlt = mlt[within_all_ranges & finite_chorus]
naive_chorus = naive_chorus[within_all_ranges & finite_chorus]

cumulative_chorus, num_points_in_each_epoch_L_bin = plot_tools.bin_3D_data(xdata = epoch,
                                                                           ydata = L, 
                                                                           zdata = naive_chorus,
                                                                           xstart = start,
                                                                           xend = end,
                                                                           xstep = epoch_bin_size,
                                                                           ystart = L_start,
                                                                           yend = L_end,
                                                                           ystep = L_bin_size)


average_chorus = cumulative_chorus / num_points_in_each_epoch_L_bin


Processing satellite: metop01
Processing satellite: metop02
Processing satellite: metop03
Processing satellite: noaa15
Processing satellite: noaa16
Processing satellite: noaa17
Processing satellite: noaa18


In [21]:
average_chorus = average_chorus + 1e-31 #Stabilize the log norm

fig, ax = plt.subplots(1, 1, figsize=(16, 9), sharex=True)

image = ax.imshow(average_chorus.T, 
                  origin = "lower",
                  extent=[start, end, L_start, L_end],
                  norm=colors.LogNorm(vmin=1, vmax=100),
                  aspect="auto",
                  interpolation="none")

image.cmap.set_under("black")

cbar = plt.colorbar(image, ax=ax, pad=0.01)

cbar.set_label("Chorus Bw (pT)\n", loc="center", labelpad=15, rotation=270)

ax.set_title("Chorus")
ax.set_ylabel("L")
ax.set_xlabel("Time")

plt.tight_layout()